In [1]:
import $exclude.`org.slf4j:slf4j-log4j12`, $ivy.`org.slf4j:slf4j-nop:1.7.21` // for cleaner logs
//import $profile.`hadoop-2.6`
import $ivy.`org.apache.spark::spark-sql:2.1.0` // adjust spark version - spark >= 2.0
//import $ivy.`org.apache.hadoop:hadoop-aws:2.6.4`
import $ivy.`org.jupyter-scala::spark:0.4.2` // for JupyterSparkSession (SparkSession aware of the jupyter-scala kernel)

import org.apache.spark._
import org.apache.spark.sql._
import jupyter.spark.session._
import Resolvers._

import $exclude.$                        , $ivy.$                            // for cleaner logs
//import $profile.`hadoop-2.6`

import $ivy.$                                   // adjust spark version - spark >= 2.0
//import $ivy.`org.apache.hadoop:hadoop-aws:2.6.4`

import $ivy.$                                // for JupyterSparkSession (SparkSession aware of the jupyter-scala kernel)


import org.apache.spark._

import org.apache.spark.sql._

import jupyter.spark.session._

import Resolvers._

In [2]:
import $ivy.`org.vegas-viz::vegas:0.3.9`
import vegas._
import vegas.data.External._

import $ivy.$                           

import vegas._

import vegas.data.External._

In [3]:
interp.resolvers() = interp.resolvers() :+ Resolver.Http(
  "isarn project",
  "https://dl.bintray.com/isarn/maven/",
  MavenPattern,
  true
)

In [4]:
import $ivy.`org.isarnproject::isarn-sketches:0.0.3.rc1` // "org.isarnproject" %% "isarn-sketches" % "0.0.2"
import org.isarnproject.sketches._

import $ivy.$                                            // "org.isarnproject" %% "isarn-sketches" % "0.0.2"

import org.isarnproject.sketches._

In [5]:
implicit class TDEnhance(td: TDigest) extends Serializable {
  def toXY: Vector[(Double, Double)] = {
    val q = (0.001 +: (0.05 to 0.95 by 0.05) :+ 0.999).toVector
    q.map(q => (td.cdfInverse(q), q))
  }
}

defined class TDEnhance

In [6]:
val spark = JupyterSparkSession.builder() // important - call this rather than SparkSession.builder()
  .jupyter() // this method must be called straightaway after builder()
  .master("spark://frclust:7077")
  .appName("notebook")
  .getOrCreate()
val sc = spark.sparkContext

log4j:WARN No appenders could be found for logger (io.netty.util.internal.logging.InternalLoggerFactory).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


spark: SparkSession = org.apache.spark.sql.SparkSession@2bd584e5

In [7]:
val raw = spark.sparkContext.parallelize(Vector.fill(1000000) { scala.util.Random.nextGaussian() }, 10)

raw: rdd.RDD[Double] = ParallelCollectionRDD[0] at parallelize at cmd6.sc:1

In [8]:
val td = raw.aggregate(TDigest.empty())(
    (td, x) => td + x,
    (td1, td2) => td1 ++ td2
)

td: TDigest = TDigest(
  0.5,
  0,
  126,
  Map(
    -4.876458933821338 -> 1.0,
    -4.752402594990833 -> 1.0,
    -4.516963979565943 -> 1.33300305145708,
    -4.511037703909945 -> 0.7334025922648455,
    -4.498892284105547 -> 0.9335943562780744,
    -4.438486676168825 -> 2.0,
    -4.274182620469876 -> 4.578080016239642,
...

In [9]:
Vegas("CDF")
    .withXY(td.toXY)
    .encodeX("x", Quant)
    .encodeY("y", Quant)
    .mark(Line)
    .show

<iframe id="frame-vegas-f3448812-6cb0-43ea-824c-b297175c166a" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "mark" : "line",
 "encoding" : {
 "x" : {
 "field" : "x",
 "type" : "quantitative"
 },
 "y" : {
 "field" : "y",
 "type" : "quantitative"
 }
 },
 "description" : "CDF",
 "data" : {
 "values" : [
 {
 "x" : -3.0963617496232008,
 "y" : 0.001
 },
 {
 "x" : -1.6516383554710787,
 "y" : 0.05
 },
 {
 "x" : -1.2798431850690153,
 "y" : 0.1
 },
 {
 "x" : -1.0378526913032835,
 "y" : 0.15000000000000002
 },
 {
 "x" : -0.8442114038769389,
 "y" : 0.2
 },
 {
 "x" : -0.677000563217571,
 "y" : 0.25
 },
 {
 "x" : -0.5231804226059978,
 "y" : 0.3
 },
 {
 "x" : -0.3831867553922784,
 "y" : 0.35
 },
 {
 "x" : -0.2552629278007744,
 "y" : 0.39999999999999997
 },
 {
 "x" : -0.12686047752487087,
 "y" : 0.44999999999999996
 },
 {
 "x" : -0.0010340336323640537,
 "y" : 0.49999999999999994
 },
 {
 "x" : 0.12447364458989851,
 "y" : 0.5499999999999999
 },
 {
 "x" : 0.26139230063967617,
 "y" : 0.6
 },
 {
 "x" : 0.37752707911859457,
 "y" : 0.65
 },
 {
 "x" : 0.523578018824731,
 "y" : 0.7000000000000001
 },
 {
 "x" : 0.6770193885751735,
 "y" : 0.7500000000000001
 },
 {
 "x" : 0.8420923267534832,
 "y" : 0.8000000000000002
 },
 {
 "x" : 1.0336296431493588,
 "y" : 0.8500000000000002
 },
 {
 "x" : 1.2832295897678208,
 "y" : 0.9000000000000002
 },
 {
 "x" : 3.0851973252646205,
 "y" : 0.999
 }
 ]
 }
}
 }
 vg.embed("#vegas-f3448812-6cb0-43ea-824c-b297175c166a", embedSpec, function(error, result) {});
 </script>
 <div id='vegas-f3448812-6cb0-43ea-824c-b297175c166a'></div>
 </body>
</html>">

In [10]:
val trainCSV = spark.read
  .format("com.databricks.spark.csv")
  .option("header", "true")
  .option("mode", "DROPMALFORMED")
  .load("/data/tox21/tox21_dense_train_orig.csv")

trainCSV: DataFrame = [_c0: string, AW: string ... 800 more fields]

In [11]:
trainCSV.columns

res10: Array[String] = Array(
  "_c0",
  "AW",
  "AWeight",
  "Arto",
  "BertzCT",
  "Chi0",
  "Chi1",
  "Chi10",
  "Chi2",
  "Chi3",
  "Chi3c",
...

In [12]:
val trainFV = trainCSV.rdd.map(_.toSeq.toVector.drop(1).map(_.asInstanceOf[String].toDouble))

trainFV: rdd.RDD[Vector[Double]] = MapPartitionsRDD[12] at map at cmd11.sc:1

In [36]:
import org.apache.spark.rdd.RDD
def tdSketchFV(fv: RDD[Vector[Double]]): Vector[TDigest] = {
    val tds = fv.aggregate(Array.empty[TDigest])(
        (tdv, xv) => {
            if (tdv.isEmpty) {
                Array.tabulate(xv.length) { j => TDigest.empty(maxDiscrete=50) + xv(j) }
            } else {
                for { j <- 0 until xv.length } { tdv(j) += xv(j) }
                tdv
            }
        },
        (tdv1, tdv2) => {
            if (tdv1.isEmpty) tdv2 else {
                for { j <- 0 until tdv1.length } { tdv1(j) ++= tdv2(j) }
                tdv1
            }
        }
    )
    tds.toVector
}

import org.apache.spark.rdd.RDD

defined function tdSketchFV

In [39]:
def synthesize(tdVec: Vector[TDigest], n: Int, partitions: Int = 2) = {
    implicit class AddSampling(td: TDigest) {
        import org.isarnproject.collections.mixmaps.nearest.Cover
        def cdfDiscreteInverse(q: Double): Double = {
            td.clusters.mCover(q * td.clusters.sum).map(n => (n.data.key, n.data.value)) match {
                case Cover(_, Some((x, _))) => x
                case Cover(Some((x, _)), None) => x
            }
        }

        def sample: Double = {
            val clust = td.clusters
            td.nclusters match {
                case 0 => 0.0
                case 1 => clust(clust.keyMin.get)
                case n if (n <= td.maxDiscrete) => cdfDiscreteInverse(scala.util.Random.nextDouble)
                case _ => td.cdfInverse(scala.util.Random.nextDouble)
            }
        }
    }
    val tdVecBC = spark.sparkContext.broadcast(tdVec)
    spark.sparkContext.parallelize(1 to n, partitions).map { _ => tdVecBC.value.map(_.sample) }
}

defined function synthesize

In [43]:
val fvSketches = tdSketchFV(trainFV)
val fvSynthetic = synthesize(fvSketches, 12000, partitions = 4)

fvSketches: Vector[TDigest] = Vector(
  TDigest(
    0.5,
    50,
    73,
    Map(
      1.0 -> 9.0,
      1.333 -> 24.0,
      1.5 -> 45.0,
      1.600489455071117 -> 33.242848970251714,
      1.667 -> 0.44314380967990985,
      1.6935383378931186 -> 67.06936934070602,
...
fvSynthetic: RDD[Vector[Double]] = MapPartitionsRDD[30] at map at cmd38.sc:22

In [44]:
fvSynthetic.first

res43: Vector[Double] = Vector(
  7435344.61895041,
  16.81669304901527,
  1.966682074011672,
  2.473779561692713,
  12.910164466288027,
  14.452480835231045,
  0.011838576903800944,
  18.939043053234677,
  5.337279655614921,
  1.887693913259665,
  0.0,
...